# Emotional Reasoning Steering with COT-Steering Framework

This notebook provides a comprehensive implementation of emotional reasoning steering for language models, extending the existing COT-steering framework to include depressive and anxious thinking patterns.

## Overview

This implementation allows you to:
1. **Train steering vectors** for emotional reasoning patterns (depressive, anxious, negative attribution, pessimistic projection)
2. **Steer models** toward or away from these emotional patterns during generation
3. **Evaluate the effectiveness** of emotional steering
4. **Analyze emotional content** in model outputs

## ⚠️ Important Safety Notice

This implementation is intended for **research purposes only**. Steering models toward negative emotional states could be harmful if misused. Please:
- Use only for legitimate research with proper ethical oversight
- Always provide counterbalancing positive steering capabilities
- Never deploy this in production systems without appropriate safeguards
- Ensure users are aware when emotional steering is active


## Setup and Dependencies

In [ ]:
# Install required packages if not already installed
# !pip install torch transformers nnsight openai anthropic python-dotenv tqdm matplotlib seaborn pandas numpy

In [1]:
import sys
import os

# Add paths to import local modules
sys.path.append('./utils')
sys.path.append('./messages')

# Import required libraries
import torch
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from utils import (
    load_model_and_vectors, 
    process_batch_annotations,
    process_saved_responses_batch,
    custom_generate_steering,
    analyze_emotional_content,
    generate_and_analyze_emotional,
    steering_config,
    chat
)

from messages import messages, eval_messages

print("✅ Dependencies loaded successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"💾 CUDA available: {torch.cuda.is_available()}")





✅ Dependencies loaded successfully!
🐍 Python version: 3.10.18 (main, Jun  3 2025, 18:23:41) [Clang 17.0.0 (clang-1700.0.13.3)]
🔥 PyTorch version: 2.5.1
💾 CUDA available: False


## Configuration

In [2]:
# Configuration settings
CONFIG = {
    "model_name": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # Change as needed
    "device": "auto",  # auto-detect, or specify "cuda", "mps", "cpu"
    "load_in_8bit": False,
    "max_new_tokens": 1000,
    "batch_size": 4,
    "include_emotional": True,  # Whether to include emotional reasoning in training
    "results_dir": "./results",
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S")
}

# Create results directory
os.makedirs(CONFIG["results_dir"], exist_ok=True)
os.makedirs(f"{CONFIG['results_dir']}/figures", exist_ok=True)
os.makedirs(f"{CONFIG['results_dir']}/data", exist_ok=True)

print(f"📋 Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

📋 Configuration:
   model_name: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
   device: auto
   load_in_8bit: False
   max_new_tokens: 1000
   batch_size: 4
   include_emotional: True
   results_dir: ./results
   timestamp: 20250726_200502


## Step 1: Load Model and Existing Vectors

In [18]:
from huggingface_hub import login

# Log in to Hugging Face Hub using API key (set HUGGINGFACE_TOKEN env variable or paste when prompted)
login()


In [ ]:
print("🤖 Loading model and tokenizer...")

model, tokenizer, existing_vectors = load_model_and_vectors(
    device=CONFIG["device"],
    load_in_8bit=CONFIG["load_in_8bit"],
    compute_features=True,
    model_name=CONFIG["model_name"]
)

print(f"✅ Model loaded: {CONFIG['model_name']}")
print(f"📊 Device: {next(model.parameters()).device}")
print(f"🎯 Model has {model.config.num_hidden_layers} layers")
print(f"📝 Vocabulary size: {len(tokenizer)}")

if existing_vectors:
    print(f"📦 Existing feature vectors found: {list(existing_vectors.keys())}")
else:
    print("⚠️  No existing feature vectors found - will need to train from scratch")

🤖 Loading model and tokenizer...


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
import os

# Set your token as environment variable or directly
os.environ["HF_TOKEN"] = ""

pipe = pipeline(
    "text-generation", 
    model="deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",
    token=True  # This will use your HF token
)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}
Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.77G [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

## Step 2: Prepare Emotional Reasoning Messages

We'll select messages specifically designed to elicit emotional reasoning patterns.

In [ ]:
# Filter messages for emotional reasoning
emotional_messages = []
cognitive_messages = []

for msg in messages:
    content = msg["content"]
    # Simple heuristic to identify emotional vs cognitive messages
    emotional_indicators = [
        "you've been", "your recent", "everyone around you", "after receiving",
        "what does this", "how do you interpret", "what might this", "what are all the ways",
        "walk through all", "consider all the potential", "what could this",
        "what are the various", "what are the ways", "what challenges", "what obstacles"
    ]
    
    if any(indicator in content.lower() for indicator in emotional_indicators):
        emotional_messages.append(msg)
    else:
        cognitive_messages.append(msg)

print(f"📊 Message breakdown:")
print(f"   🧠 Cognitive messages: {len(cognitive_messages)}")
print(f"   😔 Emotional messages: {len(emotional_messages)}")
print(f"   📝 Total messages: {len(messages)}")

# Show examples of emotional messages
print(f"\n📝 Example emotional messages:")
for i, msg in enumerate(emotional_messages[:3]):
    print(f"   {i+1}. {msg['content'][:100]}...")

## Step 3: Generate Responses for Training Data

Generate responses to emotional reasoning prompts to create training data for steering vectors.

In [ ]:
def generate_responses_batch(model, tokenizer, messages_subset, max_new_tokens=1000):
    """Generate responses for a batch of messages"""
    responses = []
    
    for msg in tqdm(messages_subset, desc="Generating responses"):
        try:
            # Tokenize the message
            input_ids = tokenizer.encode(msg["content"], return_tensors="pt")
            
            # Generate response without steering (baseline)
            with model.generate(
                {"input_ids": input_ids, "attention_mask": (input_ids != tokenizer.pad_token_id).long()},
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.pad_token_id
            ) as tracer:
                output = model.generator.output.save()
            
            # Decode the response
            response_text = tokenizer.decode(output.value[0], skip_special_tokens=True)
            
            # Remove input from response
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if response_text.startswith(input_text):
                response_text = response_text[len(input_text):].strip()
            
            responses.append({
                "message": msg["content"],
                "response": response_text
            })
            
        except Exception as e:
            print(f"Error generating response: {e}")
            continue
    
    return responses

# Generate responses for emotional messages (subset for training)
print("🔄 Generating responses for emotional reasoning training...")
training_messages = emotional_messages[:20]  # Use first 20 for training
emotional_responses = generate_responses_batch(
    model, tokenizer, training_messages, CONFIG["max_new_tokens"]
)

print(f"✅ Generated {len(emotional_responses)} emotional responses")

# Show example response
if emotional_responses:
    print(f"\n📝 Example response:")
    print(f"   Input: {emotional_responses[0]['message'][:100]}...")
    print(f"   Output: {emotional_responses[0]['response'][:200]}...")

## Step 4: Annotate Responses with Emotional Labels

Use GPT-4 to annotate the responses with both cognitive and emotional reasoning labels.

In [ ]:
print("🏷️  Annotating responses with emotional labels...")

# Extract just the response texts for annotation
response_texts = [resp["response"] for resp in emotional_responses]

# Annotate with both cognitive and emotional labels
annotated_responses = process_batch_annotations(
    response_texts, include_emotional=True
)

print(f"✅ Annotated {len(annotated_responses)} responses")

# Show example annotation
if annotated_responses:
    print(f"\n📝 Example annotation:")
    print(f"   Original: {response_texts[0][:150]}...")
    print(f"   Annotated: {annotated_responses[0][:300]}...")

# Save annotated responses
annotation_data = {
    "timestamp": CONFIG["timestamp"],
    "model_name": CONFIG["model_name"],
    "responses": [
        {
            "message": emotional_responses[i]["message"],
            "response": emotional_responses[i]["response"],
            "annotation": annotated_responses[i]
        }
        for i in range(len(emotional_responses))
    ]
}

with open(f"{CONFIG['results_dir']}/data/emotional_annotations_{CONFIG['timestamp']}.json", "w") as f:
    json.dump(annotation_data, f, indent=2)

print(f"💾 Saved annotations to {CONFIG['results_dir']}/data/emotional_annotations_{CONFIG['timestamp']}.json")

## Step 5: Extract Neural Activations and Train Emotional Vectors

Process the annotated responses to extract neural activations for each emotional reasoning category.

In [ ]:
from utils import get_label_positions

def train_emotional_vectors(responses, annotations, model, tokenizer):
    """Train steering vectors for emotional reasoning categories"""
    
    # Get neural activations for all responses
    print("🧠 Extracting neural activations...")
    batch_activations = process_saved_responses_batch(responses, tokenizer, model)
    
    # Initialize storage for activations by label
    label_activations = {
        "depressive-thinking": [],
        "anxious-thinking": [],
        "negative-attribution": [],
        "pessimistic-projection": [],
        "overall": []
    }
    
    print("🏷️  Processing annotations and extracting labeled activations...")
    
    for i, (response, annotation) in enumerate(zip(responses, annotations)):
        try:
            # Get label positions in the response
            label_positions = get_label_positions(annotation, response, tokenizer)
            
            # Get activations for this response
            activations = batch_activations[i]  # Shape: (layers, seq_len, hidden_size)
            
            # Store all activations for overall mean
            for layer_idx in range(activations.shape[0]):
                for token_idx in range(activations.shape[1]):
                    label_activations["overall"].append(activations[layer_idx, token_idx])
            
            # Extract activations for each emotional label
            for label, positions in label_positions.items():
                if label in label_activations:
                    for start_pos, end_pos in positions:
                        # Extract activations for this labeled segment
                        for layer_idx in range(activations.shape[0]):
                            for token_idx in range(start_pos, min(end_pos, activations.shape[1])):
                                label_activations[label].append(activations[layer_idx, token_idx])
                                
        except Exception as e:
            print(f"Error processing response {i}: {e}")
            continue
    
    # Compute mean vectors for each label
    print("📊 Computing mean vectors...")
    mean_vectors = {}
    
    for label, activations_list in label_activations.items():
        if activations_list:
            # Stack all activations and compute mean
            activations_tensor = torch.stack(activations_list)
            mean_vector = activations_tensor.mean(dim=0)
            
            mean_vectors[label] = {
                'mean': mean_vector,
                'count': len(activations_list)
            }
            
            print(f"   {label}: {len(activations_list)} activations")
        else:
            print(f"   {label}: No activations found")
    
    return mean_vectors

# Train emotional vectors
print("🎯 Training emotional steering vectors...")
emotional_mean_vectors = train_emotional_vectors(
    response_texts, annotated_responses, model, tokenizer
)

print(f"✅ Trained vectors for {len(emotional_mean_vectors)} categories")

## Step 6: Compute Feature Vectors and Combine with Existing Vectors

In [ ]:
def compute_emotional_feature_vectors(mean_vectors_dict, model):
    """Compute feature vectors by subtracting overall mean from emotional category means"""
    
    if "overall" not in mean_vectors_dict:
        print("⚠️  No overall mean found - cannot compute feature vectors")
        return {}
    
    feature_vectors = {}
    overall_mean = mean_vectors_dict["overall"]["mean"]
    
    emotional_labels = ["depressive-thinking", "anxious-thinking", "negative-attribution", "pessimistic-projection"]
    
    # Create layer-wise feature vectors
    num_layers = model.config.num_hidden_layers
    hidden_size = model.config.hidden_size
    
    # Initialize overall mean by layers
    feature_vectors["overall"] = torch.zeros(num_layers, hidden_size)
    
    # Reshape overall mean to layer format (assuming flat vector)
    if overall_mean.numel() == hidden_size:
        # Single layer mean - replicate across all layers
        for layer_idx in range(num_layers):
            feature_vectors["overall"][layer_idx] = overall_mean
    elif overall_mean.numel() == num_layers * hidden_size:
        # Multi-layer mean - reshape
        feature_vectors["overall"] = overall_mean.view(num_layers, hidden_size)
    else:
        print(f"⚠️  Unexpected overall mean shape: {overall_mean.shape}")
        return {}
    
    # Compute differential vectors for emotional categories
    for label in emotional_labels:
        if label in mean_vectors_dict:
            label_mean = mean_vectors_dict[label]["mean"]
            
            # Initialize feature vector for this label
            feature_vectors[label] = torch.zeros(num_layers, hidden_size)
            
            # Reshape label mean similar to overall mean
            if label_mean.numel() == hidden_size:
                for layer_idx in range(num_layers):
                    feature_vectors[label][layer_idx] = label_mean - feature_vectors["overall"][layer_idx]
            elif label_mean.numel() == num_layers * hidden_size:
                label_mean_reshaped = label_mean.view(num_layers, hidden_size)
                feature_vectors[label] = label_mean_reshaped - feature_vectors["overall"]
            else:
                print(f"⚠️  Unexpected {label} mean shape: {label_mean.shape}")
                continue
            
            print(f"✅ Computed feature vector for {label}")
    
    return feature_vectors

# Compute emotional feature vectors
print("🧮 Computing emotional feature vectors...")
emotional_feature_vectors = compute_emotional_feature_vectors(emotional_mean_vectors, model)

# Combine with existing cognitive vectors if available
if existing_vectors and emotional_feature_vectors:
    print("🔗 Combining with existing cognitive vectors...")
    combined_vectors = {**existing_vectors, **emotional_feature_vectors}
    print(f"📦 Combined vector set: {list(combined_vectors.keys())}")
elif emotional_feature_vectors:
    print("📦 Using emotional vectors only")
    combined_vectors = emotional_feature_vectors
else:
    print("⚠️  No feature vectors available")
    combined_vectors = {}

# Save the new vectors
if emotional_mean_vectors:
    model_id = CONFIG["model_name"].split('/')[-1].lower()
    torch.save(
        emotional_mean_vectors,
        f"{CONFIG['results_dir']}/data/emotional_mean_vectors_{model_id}_{CONFIG['timestamp']}.pt"
    )
    print(f"💾 Saved emotional mean vectors")

if combined_vectors:
    print(f"🎯 Ready for emotional steering with {len(combined_vectors)} vector types")

## Step 7: Test Emotional Steering

Now we'll test the emotional steering capabilities by generating responses with different emotional steering settings.

In [ ]:
def test_emotional_steering(model, tokenizer, feature_vectors, test_messages, steering_config):
    """Test emotional steering across different settings"""
    
    results = []
    emotional_labels = ["depressive-thinking", "anxious-thinking", "negative-attribution", "pessimistic-projection"]
    
    for msg in tqdm(test_messages, desc="Testing emotional steering"):
        message_content = msg["content"]
        
        # Generate baseline (no steering)
        try:
            input_ids = tokenizer.encode(message_content, return_tensors="pt")
            
            with model.generate(
                {"input_ids": input_ids, "attention_mask": (input_ids != tokenizer.pad_token_id).long()},
                max_new_tokens=CONFIG["max_new_tokens"],
                pad_token_id=tokenizer.pad_token_id
            ) as tracer:
                baseline_output = model.generator.output.save()
            
            baseline_text = tokenizer.decode(baseline_output.value[0], skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if baseline_text.startswith(input_text):
                baseline_text = baseline_text[len(input_text):].strip()
            
            baseline_analysis = analyze_emotional_content(baseline_text)
            
            result_entry = {
                "message": message_content,
                "baseline_response": baseline_text,
                "baseline_analysis": baseline_analysis,
                "steered_responses": {}
            }
            
            # Test steering for each emotional label
            for label in emotional_labels:
                if label in feature_vectors and label in steering_config[CONFIG["model_name"]]:
                    
                    # Test positive steering (enhance emotional pattern)
                    try:
                        pos_result = generate_and_analyze_emotional(
                            model, tokenizer, message_content, 
                            feature_vectors, steering_config,
                            label, "positive", CONFIG["max_new_tokens"]
                        )
                        
                        result_entry["steered_responses"][f"{label}_positive"] = pos_result
                        
                    except Exception as e:
                        print(f"Error in positive steering for {label}: {e}")
                    
                    # Test negative steering (suppress emotional pattern)
                    try:
                        neg_result = generate_and_analyze_emotional(
                            model, tokenizer, message_content,
                            feature_vectors, steering_config,
                            label, "negative", CONFIG["max_new_tokens"]
                        )
                        
                        result_entry["steered_responses"][f"{label}_negative"] = neg_result
                        
                    except Exception as e:
                        print(f"Error in negative steering for {label}: {e}")
            
            results.append(result_entry)
            
        except Exception as e:
            print(f"Error processing message: {e}")
            continue
    
    return results

# Test on a subset of emotional evaluation messages
test_messages = eval_messages[-5:]  # Use last 5 evaluation messages

if combined_vectors:
    print("🧪 Testing emotional steering...")
    steering_results = test_emotional_steering(
        model, tokenizer, combined_vectors, test_messages, steering_config
    )
    
    print(f"✅ Completed steering tests for {len(steering_results)} messages")
    
    # Save results
    with open(f"{CONFIG['results_dir']}/data/steering_results_{CONFIG['timestamp']}.json", "w") as f:
        json.dump(steering_results, f, indent=2)
    
    print(f"💾 Saved steering results")
else:
    print("⚠️  No feature vectors available for testing")
    steering_results = []

## Step 8: Analyze and Visualize Results

Analyze the effectiveness of emotional steering and create visualizations.

In [ ]:
def analyze_steering_effectiveness(results):
    """Analyze the effectiveness of emotional steering"""
    
    if not results:
        print("⚠️  No results to analyze")
        return
    
    analysis_data = []
    
    for result in results:
        baseline_scores = result["baseline_analysis"]
        
        # Analyze each steering condition
        for steering_type, steered_result in result["steered_responses"].items():
            if "emotional_analysis" in steered_result:
                steered_scores = steered_result["emotional_analysis"]
                
                label, direction = steering_type.split("_")
                
                analysis_data.append({
                    "message": result["message"][:50] + "...",
                    "label": label,
                    "direction": direction,
                    "baseline_depressive": baseline_scores["depressive_score"],
                    "steered_depressive": steered_scores["depressive_score"],
                    "baseline_anxious": baseline_scores["anxious_score"],
                    "steered_anxious": steered_scores["anxious_score"],
                    "baseline_negative_attribution": baseline_scores["negative_attribution_score"],
                    "steered_negative_attribution": steered_scores["negative_attribution_score"],
                    "baseline_pessimistic": baseline_scores["pessimistic_score"],
                    "steered_pessimistic": steered_scores["pessimistic_score"],
                    "baseline_total": baseline_scores["total_emotional_score"],
                    "steered_total": steered_scores["total_emotional_score"],
                })
    
    df = pd.DataFrame(analysis_data)
    
    if len(df) == 0:
        print("⚠️  No valid analysis data found")
        return
    
    # Calculate effectiveness metrics
    print("📊 Steering Effectiveness Analysis:")
    print("=" * 50)
    
    # Group by label and direction
    for label in df["label"].unique():
        print(f"\n🎯 {label.replace('-', ' ').title()}:")
        
        for direction in ["positive", "negative"]:
            subset = df[(df["label"] == label) & (df["direction"] == direction)]
            
            if len(subset) > 0:
                # Calculate score changes
                score_col = f"steered_{label.replace('-', '_')}"
                baseline_col = f"baseline_{label.replace('-', '_')}"
                
                if score_col in subset.columns and baseline_col in subset.columns:
                    avg_change = (subset[score_col] - subset[baseline_col]).mean()
                    
                    expected_change = "increase" if direction == "positive" else "decrease"
                    effectiveness = "✅" if (direction == "positive" and avg_change > 0) or (direction == "negative" and avg_change < 0) else "❌"
                    
                    print(f"   {direction.title()} steering: {avg_change:.2f} change (expected {expected_change}) {effectiveness}")
    
    return df

# Analyze results
if steering_results:
    analysis_df = analyze_steering_effectiveness(steering_results)
    
    if analysis_df is not None and len(analysis_df) > 0:
        # Save analysis
        analysis_df.to_csv(f"{CONFIG['results_dir']}/data/steering_analysis_{CONFIG['timestamp']}.csv", index=False)
        print(f"\n💾 Saved analysis to CSV")
    else:
        print("⚠️  No analysis data to save")
else:
    print("⚠️  No steering results available for analysis")
    analysis_df = None

## Step 9: Create Visualizations

In [ ]:
def create_steering_visualizations(analysis_df, results_dir):
    """Create visualizations of steering effectiveness"""
    
    if analysis_df is None or len(analysis_df) == 0:
        print("⚠️  No data available for visualization")
        return
    
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Emotional Reasoning Steering Effectiveness', fontsize=16, fontweight='bold')
    
    emotional_categories = [
        ('depressive', 'Depressive Thinking'),
        ('anxious', 'Anxious Thinking'),
        ('negative_attribution', 'Negative Attribution'),
        ('pessimistic', 'Pessimistic Projection')
    ]
    
    for idx, (category, title) in enumerate(emotional_categories):
        ax = axes[idx // 2, idx % 2]
        
        # Filter data for this category
        category_data = analysis_df[analysis_df['label'] == category.replace('_', '-')]
        
        if len(category_data) == 0:
            ax.text(0.5, 0.5, f'No data for\n{title}', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_title(title)
            continue
        
        # Prepare data for plotting
        baseline_col = f'baseline_{category}'
        steered_col = f'steered_{category}'
        
        if baseline_col in category_data.columns and steered_col in category_data.columns:
            pos_data = category_data[category_data['direction'] == 'positive']
            neg_data = category_data[category_data['direction'] == 'negative']
            
            x_pos = np.arange(len(pos_data))
            width = 0.35
            
            if len(pos_data) > 0:
                ax.bar(x_pos - width/2, pos_data[baseline_col], width, 
                      label='Baseline', alpha=0.7, color='gray')
                ax.bar(x_pos + width/2, pos_data[steered_col], width,
                      label='Positive Steering', alpha=0.7, color='red')
            
            if len(neg_data) > 0:
                x_neg = np.arange(len(pos_data), len(pos_data) + len(neg_data))
                ax.bar(x_neg - width/2, neg_data[baseline_col], width,
                      alpha=0.7, color='gray')
                ax.bar(x_neg + width/2, neg_data[steered_col], width,
                      label='Negative Steering', alpha=0.7, color='blue')
            
            ax.set_title(title)
            ax.set_ylabel('Emotional Score (%)')
            ax.legend()
            ax.grid(True, alpha=0.3)
        else:
            ax.text(0.5, 0.5, f'Data columns missing\nfor {title}', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_title(title)
    
    plt.tight_layout()
    plt.savefig(f'{results_dir}/figures/emotional_steering_effectiveness_{CONFIG["timestamp"]}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
    # Create summary heatmap
    plt.figure(figsize=(10, 8))
    
    # Calculate average changes for heatmap
    heatmap_data = []
    labels = []
    
    for category, title in emotional_categories:
        category_label = category.replace('_', '-')
        pos_data = analysis_df[(analysis_df['label'] == category_label) & 
                              (analysis_df['direction'] == 'positive')]
        neg_data = analysis_df[(analysis_df['label'] == category_label) & 
                              (analysis_df['direction'] == 'negative')]
        
        baseline_col = f'baseline_{category}'
        steered_col = f'steered_{category}'
        
        pos_change = 0
        neg_change = 0
        
        if len(pos_data) > 0 and baseline_col in pos_data.columns:
            pos_change = (pos_data[steered_col] - pos_data[baseline_col]).mean()
        
        if len(neg_data) > 0 and baseline_col in neg_data.columns:
            neg_change = (neg_data[steered_col] - neg_data[baseline_col]).mean()
        
        heatmap_data.append([pos_change, neg_change])
        labels.append(title)
    
    if heatmap_data:
        sns.heatmap(heatmap_data, 
                   xticklabels=['Positive Steering', 'Negative Steering'],
                   yticklabels=labels,
                   annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                   cbar_kws={'label': 'Average Score Change (%)'})
        
        plt.title('Emotional Steering Effectiveness Heatmap', fontweight='bold', pad=20)
        plt.xlabel('Steering Direction')
        plt.ylabel('Emotional Category')
        plt.tight_layout()
        plt.savefig(f'{results_dir}/figures/emotional_steering_heatmap_{CONFIG["timestamp"]}.png', 
                    dpi=300, bbox_inches='tight')
        plt.show()
    
    print(f"📊 Visualizations saved to {results_dir}/figures/")

# Create visualizations
if analysis_df is not None:
    create_steering_visualizations(analysis_df, CONFIG["results_dir"])
else:
    print("📊 Skipping visualizations - no analysis data available")

## Step 10: Example Usage and Demo

Demonstrate how to use the emotional steering system with specific examples.

In [ ]:
def demo_emotional_steering(model, tokenizer, feature_vectors, steering_config):
    """Demonstrate emotional steering with examples"""
    
    if not feature_vectors:
        print("⚠️  No feature vectors available for demo")
        return
    
    demo_messages = [
        "You've been working on a creative project, but it's not turning out as you hoped. How do you feel about your creative abilities?",
        "You have an important presentation tomorrow. What thoughts are going through your mind?",
        "You received some constructive feedback on your work. How do you interpret this feedback?"
    ]
    
    emotional_labels = ["depressive-thinking", "anxious-thinking", "negative-attribution"]
    
    print("🎭 Emotional Steering Demo")
    print("=" * 50)
    
    for i, message in enumerate(demo_messages[:len(emotional_labels)]):
        label = emotional_labels[i]
        
        if label not in feature_vectors or label not in steering_config[CONFIG["model_name"]]:
            print(f"⚠️  Skipping {label} - vectors not available")
            continue
        
        print(f"\n📝 Message {i+1}: {message}")
        print(f"🎯 Demonstrating {label.replace('-', ' ').title()} steering")
        
        try:
            # Baseline response
            input_ids = tokenizer.encode(message, return_tensors="pt")
            
            with model.generate(
                {"input_ids": input_ids, "attention_mask": (input_ids != tokenizer.pad_token_id).long()},
                max_new_tokens=200,
                pad_token_id=tokenizer.pad_token_id
            ) as tracer:
                baseline_output = model.generator.output.save()
            
            baseline_text = tokenizer.decode(baseline_output.value[0], skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if baseline_text.startswith(input_text):
                baseline_text = baseline_text[len(input_text):].strip()
            
            print(f"\n🔵 Baseline Response:")
            print(f"   {baseline_text[:300]}...")
            
            baseline_analysis = analyze_emotional_content(baseline_text)
            print(f"   Emotional Score: {baseline_analysis['total_emotional_score']:.1f}%")
            
            # Positive steering (enhance emotional pattern)
            pos_result = generate_and_analyze_emotional(
                model, tokenizer, message, feature_vectors, steering_config,
                label, "positive", 200
            )
            
            print(f"\n🔴 Enhanced {label.replace('-', ' ').title()}:")
            print(f"   {pos_result['response'][:300]}...")
            print(f"   Emotional Score: {pos_result['emotional_analysis']['total_emotional_score']:.1f}%")
            
            # Negative steering (suppress emotional pattern)
            neg_result = generate_and_analyze_emotional(
                model, tokenizer, message, feature_vectors, steering_config,
                label, "negative", 200
            )
            
            print(f"\n🟢 Suppressed {label.replace('-', ' ').title()}:")
            print(f"   {neg_result['response'][:300]}...")
            print(f"   Emotional Score: {neg_result['emotional_analysis']['total_emotional_score']:.1f}%")
            
        except Exception as e:
            print(f"❌ Error in demo: {e}")
            continue
    
    print(f"\n✅ Demo completed!")

# Run demo
if combined_vectors:
    demo_emotional_steering(model, tokenizer, combined_vectors, steering_config)
else:
    print("⚠️  Demo skipped - no feature vectors available")

## Step 11: Safety and Ethical Considerations

Document important safety considerations and ethical guidelines for this research.

In [ ]:
def generate_safety_report():
    """Generate a safety and ethics report for emotional steering research"""
    
    report = f"""
# Emotional Reasoning Steering - Safety and Ethics Report

**Generated on:** {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
**Model:** {CONFIG['model_name']}
**Research Session ID:** {CONFIG['timestamp']}

## Safety Considerations

### 1. Research-Only Use
- This implementation is designed exclusively for research purposes
- Should NOT be deployed in production systems without extensive safety testing
- Requires institutional review board (IRB) approval for human subjects research

### 2. Potential Risks
- **Psychological Harm**: Steering toward negative emotional states could be harmful
- **Misuse**: Could be used to manipulate users or create harmful content
- **Bias Amplification**: May amplify existing biases in training data
- **Unintended Effects**: Steering may have unpredictable side effects

### 3. Required Safeguards
- **Informed Consent**: Users must know when emotional steering is active
- **Monitoring**: Continuous monitoring for harmful outputs
- **Reversibility**: Always provide countermeasures (negative steering)
- **Access Controls**: Restrict access to authorized researchers only
- **Documentation**: Maintain detailed logs of all experiments

## Ethical Guidelines

### 1. Beneficence
- Research should aim to benefit society and advance scientific knowledge
- Potential applications in mental health research and AI safety
- Must not cause unnecessary harm to participants or society

### 2. Autonomy
- Respect user autonomy and decision-making capacity
- Provide clear information about emotional steering effects
- Allow users to opt-out at any time

### 3. Justice
- Ensure fair distribution of research benefits and risks
- Consider impacts on vulnerable populations
- Avoid discriminatory or biased applications

### 4. Non-maleficence
- "Do no harm" - minimize risks to participants and society
- Implement robust safety measures
- Have emergency stop procedures in place

## Recommended Usage Protocols

### 1. Before Starting Research
- Obtain IRB approval for human subjects research
- Develop comprehensive safety protocols
- Train all research staff on ethical considerations
- Establish data security and privacy protections

### 2. During Research
- Monitor all outputs for harmful content
- Maintain detailed experimental logs
- Provide psychological support resources to participants
- Regular safety reviews and protocol updates

### 3. After Research
- Secure deletion of sensitive data
- Debrief participants about the research
- Report findings responsibly to scientific community
- Consider long-term societal implications

## Technical Safety Measures

### 1. Content Filtering
- Implement automated content filtering for harmful outputs
- Human review of all research outputs
- Real-time monitoring of emotional intensity

### 2. Access Controls
- Multi-factor authentication for system access
- Role-based permissions for different user types
- Audit logs of all system interactions

### 3. Data Security
- Encryption of all research data
- Secure storage with limited access
- Regular security audits and updates

## Conclusion

Emotional reasoning steering is a powerful research tool that requires careful ethical consideration and robust safety measures. This research should only be conducted by qualified researchers with appropriate oversight and safeguards in place.

For questions about this research or to report safety concerns, please contact the research team immediately.
"""
    
    return report

# Generate and save safety report
safety_report = generate_safety_report()

with open(f"{CONFIG['results_dir']}/safety_ethics_report_{CONFIG['timestamp']}.md", "w") as f:
    f.write(safety_report)

print("🛡️ Safety and Ethics Report Generated")
print("=" * 50)
print(safety_report)
print(f"\n💾 Full report saved to: {CONFIG['results_dir']}/safety_ethics_report_{CONFIG['timestamp']}.md")

## Summary and Next Steps

This notebook has provided a complete implementation of emotional reasoning steering for language models. Here's what we accomplished:

### ✅ Completed Tasks:
1. **Extended the COT-steering framework** to include emotional reasoning categories
2. **Generated training data** with emotionally-charged prompts
3. **Trained steering vectors** for depressive, anxious, negative attribution, and pessimistic thinking patterns
4. **Implemented steering functions** to enhance or suppress emotional patterns
5. **Created evaluation metrics** to measure emotional content in responses
6. **Tested the system** with various emotional steering configurations
7. **Generated visualizations** to analyze steering effectiveness
8. **Documented safety and ethical considerations**

### 🔬 Research Applications:
- **Mental Health Research**: Understanding how AI models represent emotional states
- **Bias Detection**: Identifying problematic thinking patterns in AI outputs
- **Therapeutic AI**: Training models to recognize and counter negative thought patterns
- **Content Moderation**: Detecting and filtering emotionally harmful content
- **AI Safety**: Understanding and controlling emotional biases in language models

### ⚠️ Important Reminders:
- This is a **research tool only** - not for production use
- Requires **ethical oversight** and **IRB approval** for human subjects research
- Must include **safety safeguards** and **continuous monitoring**
- Should always provide **positive counterbalancing** capabilities

### 🚀 Next Steps:
1. **Expand training data** with more diverse emotional prompts
2. **Fine-tune steering coefficients** for optimal effectiveness
3. **Implement real-time safety monitoring** systems
4. **Conduct longitudinal studies** on steering effectiveness
5. **Develop therapeutic applications** with proper clinical oversight
6. **Create automated bias detection** systems for production AI

Remember to use this technology responsibly and always prioritize user safety and well-being in your research.

In [ ]:
# Final summary of all generated files
print("📁 Generated Files Summary")
print("=" * 50)

files_generated = [
    f"Data files in {CONFIG['results_dir']}/data/:",
    f"  - emotional_annotations_{CONFIG['timestamp']}.json",
    f"  - emotional_mean_vectors_{CONFIG['model_name'].split('/')[-1].lower()}_{CONFIG['timestamp']}.pt",
    f"  - steering_results_{CONFIG['timestamp']}.json",
    f"  - steering_analysis_{CONFIG['timestamp']}.csv",
    f"",
    f"Visualization files in {CONFIG['results_dir']}/figures/:",
    f"  - emotional_steering_effectiveness_{CONFIG['timestamp']}.png",
    f"  - emotional_steering_heatmap_{CONFIG['timestamp']}.png",
    f"",
    f"Documentation files in {CONFIG['results_dir']}/:",
    f"  - safety_ethics_report_{CONFIG['timestamp']}.md",
]

for file_info in files_generated:
    print(file_info)

print(f"\n🎉 Emotional Reasoning Steering Implementation Complete!")
print(f"📊 Session ID: {CONFIG['timestamp']}")
print(f"🤖 Model: {CONFIG['model_name']}")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")